In [44]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from keras.models import load_model
from recognize_numbers import recognize


In [45]:
def load_data(training_data_directory):
    print("Loading data")
    file_name = os.path.join(training_data_directory, "new_test_data_a03.npy")
    training_data1 = np.load(file_name)
    file_name = os.path.join(training_data_directory, "training_data-1_1.npy")
    training_data2 = np.load(file_name)
    file_name = os.path.join(training_data_directory, "test_data-1.npy")
    training_data3 = np.load(file_name)

    return np.concatenate((training_data1, training_data2, training_data3))


training_data = load_data(r"E:\Trackmania Data\training_data_new")
print("Data with length", len(training_data), "loaded")

Loading data


Data with length 4462 loaded


Split images of velocities in three parts (3 digits):

In [46]:
velocity_images = training_data[:, 1]
img_digits = [Image.new("RGB", (18, 32)) for _ in range(3 * len(velocity_images))] 
for i in range(len(velocity_images)):
    img_digits[3 * i].paste(Image.fromarray(velocity_images[i][:, :18]))
    img_digits[3 * i + 1].paste(Image.fromarray(velocity_images[i][:, 17:35]))
    img_digits[3 * i + 2].paste(Image.fromarray(velocity_images[i][:, 35:53]))
img_digits = [np.asarray(e) for e in img_digits]

Determine each digit individually:

In [47]:
velo_recognition_model = load_model(r"C:\Users\mivog\PycharmProjects\driving ai trackmania\velocity_recognition-100.h5")
digits_with_confidence_value = recognize(np.array(img_digits)/255, velo_recognition_model)

Recombine digits to a 3 digits number:

In [48]:
velocities = [int('{}{}{}'.format(digits_with_confidence_value[3 * i][0], digits_with_confidence_value[3 * i + 1][0],
                                  digits_with_confidence_value[3 * i + 2][0]))
              for i in range(len(velocity_images))]


In [49]:
min(velocities)

0

In [50]:
max(velocities)

484

Encode key presses into array of length 4 like this: "WA" --> [1, 0, 1, 0]

In [51]:
key_presses = training_data[:, 2]


def one_hot(keys):
    key_dict = {
        "W": 0,
        "S": 1,
        "A": 2,
        "D": 3
    }
    ar = [0] * 4
    for key in keys:
        ar[key_dict[key]] = 1
    return ar


key_presses_one_hot = [one_hot(e) for e in key_presses]


In [52]:
da = pd.DataFrame({'Velocities': pd.Series(velocities), "W": pd.Series([e[0] for e in key_presses_one_hot]),
                   "S": pd.Series([e[1] for e in key_presses_one_hot]),
                   "A": pd.Series([e[2] for e in key_presses_one_hot]),
                   "D": pd.Series([e[3] for e in key_presses_one_hot])})


In [53]:
plt.imshow(training_data[0, 0])
plt.show()


In [54]:
plt.plot(velocities)
plt.show()

In [55]:
velocities_diff = np.diff(velocities)


In [56]:
plt.plot(velocities_diff)
# plt.ylim([-5, 5])
plt.show()

In [57]:
np.mean(velocities_diff)

0.08271687962340282

Define the bins for the velocity values:

In [58]:
# n, bin_edges, _ = plt.hist(velocities, 'auto')
bin_edges = np.array(np.mat('1. 28.6875 56.375 84.0625 111.75 139.4375 167.125 194.8125 222.5 250.1875 277.875 305.5625 333.25 360.9375 388.625 416.3125 444')).flatten()
plt.hist(velocities, bin_edges)
number_bins = len(bin_edges) - 1
print("number of bins:", number_bins)
print(bin_edges)
plt.show()

number of bins: 16
[  1.      28.6875  56.375   84.0625 111.75   139.4375 167.125  194.8125
 222.5    250.1875 277.875  305.5625 333.25   360.9375 388.625  416.3125
 444.    ]


In [59]:
bin_ind = np.digitize(velocities, bin_edges) - 1
for k, ind in enumerate(bin_ind):
    if ind == number_bins:
        bin_ind[k] -= 1 
velocities_binned_one_hot = []
for ind in bin_ind:
    one_hot_vector = np.zeros(number_bins)
    one_hot_vector[ind] = 1 
    velocities_binned_one_hot.append(one_hot_vector)

In [60]:
da = pd.DataFrame({'Velocities': pd.Series(velocities), 
                   "W": pd.Series([e[0] for e in key_presses_one_hot]),
                   "S": pd.Series([e[1] for e in key_presses_one_hot]),
                   "A": pd.Series([e[2] for e in key_presses_one_hot]),
                   "D": pd.Series([e[3] for e in key_presses_one_hot]),
                   "Velocities_binned": pd.Series(np.digitize(velocities, bin_edges))})


In [61]:
da.corr()

,Velocities,W,S,A,D,Velocities_binned
Velocities,1.000000,0.022579,NaN,0.219879,-0.129247,0.997188
W,0.022579,1.000000,NaN,-0.109149,0.035987,0.019946
S,NaN,NaN,NaN,NaN,NaN,NaN
A,0.219879,-0.109149,NaN,1.000000,-0.108124,0.225013
D,-0.129247,0.035987,NaN,-0.108124,1.000000,-0.124611
Velocities_binned,0.997188,0.019946,NaN,0.225013,-0.124611,1.000000


In [62]:
training_data[:, 2].shape

(4462,)

In [63]:
screenshots = training_data[:, 0]

In [64]:
import cv2

In [65]:
print(type(screenshots))
print(type(velocities_binned_one_hot))
print(type(key_presses_one_hot))

<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>


In [66]:
screenshots.shape

(4462,)

Create mirror images with mirrored key presses:

In [67]:
screenshots_mirrored = []
unchanged_velocities = []
changed_key_presses = []
for k, im in enumerate(screenshots):
    screenshots_mirrored.append(cv2.flip(im, flipCode=1))
    press = key_presses_one_hot[k]
    new_press = [press[0], press[1], 0, 0]
    if press[2] == 1:
        new_press[2] = 0
        new_press[3] = 1
    elif press[3] == 1:
        new_press[3] = 0
        new_press[2] = 1
    changed_key_presses.append(new_press)
    unchanged_velocities.append(velocities_binned_one_hot[k])

In [68]:
np.array(screenshots_mirrored).shape

(4462, 40, 150, 3)

In [69]:
screenshots.shape


(4462,)

In [70]:
screenshots_copy = screenshots.copy()

In [71]:
screenshots = np.empty((screenshots_copy.shape[0], screenshots_copy[0].shape[0], screenshots_copy[0].shape[1], screenshots_copy[0].shape[2]))

In [72]:
for k in range(len(screenshots_copy)):
    screenshots[k] = screenshots_copy[k]

In [73]:
screenshots = np.concatenate((screenshots, np.array(screenshots_mirrored)))
velocities_binned_one_hot = velocities_binned_one_hot + unchanged_velocities
key_presses_one_hot = key_presses_one_hot + changed_key_presses

In [74]:
save_name = "gesamt_processed"
# save_name = "training_binned"
file_path = r"E:\Trackmania Data\training_data_new\\" + save_name + ".npy"

In [75]:
np.array(velocities_binned_one_hot[k]).shape

(16,)

In [76]:
data_points = np.array([np.array([screenshots[k], np.array(velocities_binned_one_hot[k]), np.array(key_presses_one_hot[k])]) for k in range(len(screenshots))])

In [77]:
data_points.shape

(8924, 3)

In [78]:
np.save(file_path, data_points)

In [79]:
# import random as r
# sample = r.choices(data_points, k=500)
# for k, nr in enumerate(sample):
#     description = "Key pressed: {}".format(nr[2])
#     plt.imshow(nr[0].astype('uint8'))
#     plt.xlabel(description)
#     save_path = r"E:\Trackmania Data\training_data_new\test_images_mirror\\" + str(k) + ".png" 
#     plt.savefig(save_path)